In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
import os
import requests
import pandas

from rdkit import Chem
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import HTML

from example_helpers import show_predictions, no_wrap_div, get_dataset, process_fame_results, style_dataframe

MDSTUDIO_URL = 'http://mdstudio-smartcyp.dev.openrisknet.org/'
FAME_URL = 'http://fame3.dev.openrisknet.org/'

data = get_dataset()

## SOM prediction comparing Docking, SMARTCyp and Fame

In [5]:
for case_name, case_data in data.items():
    
    # Get SMARTCyp prediction
    response = requests.post('{0}/som_prediction'.format(MDSTUDIO_URL),
                             files={'ligand_file': case_data['mol2']}, data={'cyp': case_data['cyp']}) 
    response_df = pandas.DataFrame.from_dict(response.json(), orient='index')
    
    # Get FAME3 prediction
    response = requests.post('{0}/predictFromFiles'.format(FAME_URL), files={'files': case_data['sdf']},
                             data={'includeMDL': False}, headers={"accept" : "application/json"})
    response_json = response.json()
    response_df, famecutoff = process_fame_results(response_json, response_df)
    response_df_prob = response_df[['Docking', 'SMARTCyp', 'Fame']].fillna(0)
    
    # Draw molecule
    rdmol = Chem.rdmolfiles.MolFromMol2File('{0}.mol2'.format(case_name))
    dock_svg = show_predictions(rdmol, response_df_prob['Docking'])
    smart_svg = show_predictions(rdmol, response_df_prob['SMARTCyp'])
    fame_svg = show_predictions(rdmol, response_df_prob['Fame'], cutoff=famecutoff)
    
    display(HTML('<h3 align="center">{0}</h3>'.format(case_name)))
    display(HTML(no_wrap_div.format(dock_svg, smart_svg, fame_svg)))
    display(style_dataframe(response_df_prob, fame_cutoff=famecutoff))

,Docking,SMARTCyp,Fame
C.1,0.625,0.424,0.068
C.10,0.562,0,0.092
C.12,0.688,1,0.372
C.13,0.688,1,0.372
C.14,0.188,0.599,0.088
C.16,0.188,0.517,0.12
C.2,0.438,0.582,0.432
C.3,0.5,0.424,0.068
C.4,0.5,0.394,0.028
C.5,0,0,0


,Docking,SMARTCyp,Fame
C.10,0.071,0.672,0
C.11,0.714,0.842,0
C.12,0.286,0.673,0
C.13,0.643,0.726,0
C.14,0,0,0
C.15,0.429,0.744,0
C.16,0,0,0
C.17,0.214,0.65,0
C.18,0.929,0.765,0.795
C.19,0.286,0.766,0.806


,Docking,SMARTCyp,Fame
C.1,0,0,0.02
C.10,0.053,0.393,0.004
C.11,0.421,0.393,0.004
C.12,0.316,0.393,0.004
C.14,1,0.412,0.196
C.15,0.105,0.413,0.036
C.16,0.053,0.423,0.012
C.17,0.053,0.423,0.012
C.18,0.526,0.423,0.012
C.19,0.474,0.423,0.012
